# Traitement des données

In [ ]:
import numpy as np
import pylab as pl
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd

X_train = pd.read_csv('X_train_HI5.csv')
X_test = pd.read_csv('X_test_HI5.csv')

y_train = X_train['piezo_groundwater_level_category']
X_train = X_train.drop(['piezo_groundwater_level_category'], axis=1) 

## I. Cohérence des types

### 1.1 Piezo variables

In [ ]:
# Filtrer les colonnes commençant par "piezo"
piezo_columns = [col for col in X_train.columns if col.startswith('piezo')]

# Vérifier les types des colonnes sélectionnées
piezo_types = X_train[piezo_columns].dtypes

# Afficher les résultats
piezo_types


**Dates (actuellement object) :**
Les colonnes piezo_station_update_date et piezo_measurement_date sont des dates, mais elles sont au format object.
Action : Convertir ces colonnes en type datetime.

In [ ]:
X_train['piezo_station_update_date'] = X_train['piezo_station_update_date'].str.replace(r'CEST', '', regex=True).str.strip()
X_train['piezo_measurement_date'] = X_train['piezo_measurement_date'].str.replace(r'CEST', '', regex=True).str.strip()
X_train['piezo_station_update_date'] = pd.to_datetime(X_train['piezo_station_update_date'], errors='coerce')
X_train['piezo_measurement_date'] = pd.to_datetime(X_train['piezo_measurement_date'], errors='coerce')

X_test['piezo_station_update_date'] = X_test['piezo_station_update_date'].str.replace(r'CEST', '', regex=True).str.strip()
X_test['piezo_measurement_date'] = X_test['piezo_measurement_date'].str.replace(r'CEST', '', regex=True).str.strip()    
X_test['piezo_station_update_date'] = pd.to_datetime(X_test['piezo_station_update_date'], errors='coerce')  
X_test['piezo_measurement_date'] = pd.to_datetime(X_test['piezo_measurement_date'], errors='coerce')    


**Codes et identifiants (actuellement object) :** 

Colonnes concernées :  

piezo_station_department_code, piezo_station_bdlisa_codes, piezo_station_bss_code, piezo_station_bss_id, piezo_bss_code.  

Ces colonnes contiennent des codes ou identifiants. Si elles ne sont pas utilisées comme valeurs numériques, elles peuvent rester au format object. Sinon, vérifier s'il est pertinent de les convertir en category pour économiser de la mémoire.  

Action : Convertir en category si elles ne nécessitent pas d'opérations numériques.

In [ ]:
cols_to_category = [
    'piezo_station_department_code', 'piezo_station_bdlisa_codes', 
    'piezo_station_bss_code', 'piezo_station_bss_id', 'piezo_bss_code'
]
X_train[cols_to_category] = X_train[cols_to_category].astype('category')
X_test[cols_to_category] = X_test[cols_to_category].astype('category')



**Catégories qualitatives (actuellement object) :**  

Colonnes concernées :  

piezo_station_department_name, piezo_station_commune_name, piezo_obtention_mode, piezo_status, piezo_qualification, piezo_continuity_name, piezo_producer_name, piezo_measure_nature_code, piezo_measure_nature_name.  

Ces colonnes semblent représenter des catégories qualitatives (ex. : nom de département, nature de la mesure).  

Action : Convertir en type category si elles sont destinées à être utilisées pour le regroupement, l'analyse ou la modélisation.

In [ ]:
qualitative_cols = [
    'piezo_station_department_name', 'piezo_station_commune_name', 
    'piezo_obtention_mode', 'piezo_status', 'piezo_qualification', 
    'piezo_continuity_name', 'piezo_producer_name', 
    'piezo_measure_nature_code', 'piezo_measure_nature_name'
]
X_train[qualitative_cols] = X_train[qualitative_cols].astype('category')
X_test[qualitative_cols] = X_test[qualitative_cols].astype('category')



**Colonnes numériques (déjà float64)**

Colonnes concernées :   

piezo_station_investigation_depth, piezo_station_altitude, piezo_station_longitude, piezo_station_latitude, piezo_continuity_code, piezo_producer_code.   

Ces colonnes semblent correctement typées. Vérifie cependant la présence d'erreurs ou de valeurs aberrantes (NaN, valeurs extrêmes).   

Action : Analyser les valeurs aberrantes et les NaN pour décider d'un traitement adapté.

In [ ]:
print(X_train[['piezo_station_investigation_depth', 'piezo_station_altitude']].describe())


### 1.2 Meteo variables

In [ ]:
def print_full(x):
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 2000)
    pd.set_option('display.float_format', '{:20,.4f}'.format)
    pd.set_option('display.max_colwidth', None)
    print(x)
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')
    pd.reset_option('display.width')
    pd.reset_option('display.float_format')
    pd.reset_option('display.max_colwidth')

In [ ]:
# Filtrer les colonnes commençant par "meteo_"
meteo_columns = [col for col in X_train.columns if col.startswith('meteo_')]

# Vérifier les types des colonnes sélectionnées
meteo_types = X_train[meteo_columns].dtypes

# Calculer les statistiques descriptives pour les colonnes quantitatives
print_full(meteo_types)



**Dates (meteo_date)**

Actuellement au format object, ce qui limite les manipulations.  

Action : Convertir en type datetime pour permettre les analyses temporelles.

In [ ]:
X_train['meteo_date'] = pd.to_datetime(X_train['meteo_date'], errors='coerce')
X_test['meteo_date'] = pd.to_datetime(X_test['meteo_date'], errors='coerce')



**Identifiants et noms (meteo_id, meteo_name)**

meteo_id (station ID) : Correctement typé en int64.  

meteo_name (nom de la station) : Peut être converti en type category si utilisé pour le regroupement ou l'analyse.  

Action : Convertir meteo_name en category.

In [ ]:
X_train['meteo_name'] = X_train['meteo_name'].astype('category')
X_test['meteo_name'] = X_test['meteo_name'].astype('category')



**3. Variables booléennes encodées en float64**

Colonnes : meteo_if_snow, meteo_if_fog, meteo_if_thunderstorm, etc. 

Ces colonnes semblent représenter des indicateurs booléens (présence/absence). 

Action : Convertir en type bool.

In [ ]:
bool_columns = [
    'meteo_if_snow', 'meteo_if_fog', 'meteo_if_thunderstorm', 'meteo_if_sleet', 'meteo_if_hail',
    'meteo_if_dew', 'meteo_if_black_ice', 'meteo_if_snow_ground', 'meteo_if_frost', 
    'meteo_if_smoke', 'meteo_if_mist', 'meteo_if_lightning'
]
X_train[bool_columns] = X_train[bool_columns].astype('bool')
X_test[bool_columns] = X_test[bool_columns].astype('bool')



**Variables quantitatives (float64)**

Les variables comme meteo_temperature_min, meteo_radiation, etc., sont correctement typées. 

Action : Vérifier les valeurs aberrantes et remplacer les NaN par des stratégies spécifiques (moyenne, interpolation, etc.).

In [ ]:
# Identifier les colonnes quantitatives
quantitative_vars = X_train[meteo_columns].select_dtypes(include=['float64', 'int64']).columns
print(quantitative_vars)

# Calculer les statistiques descriptives pour les colonnes quantitatives
print_full(X_train[quantitative_vars].describe())


### 1.3 Hydro variables

In [ ]:
# Filtrer les colonnes commençant par "hydro_"
hydro_columns = [col for col in X_train.columns if col.startswith('hydro_')]

# Vérifier les types des colonnes sélectionnées
hydro_types = X_train[hydro_columns].dtypes

# Calculer les statistiques descriptives pour les colonnes quantitatives
hydro_quantitative = X_train[hydro_columns].select_dtypes(include=['float64', 'int64']).describe()

# Afficher les résultats des types et des statistiques
print_full(hydro_types)


**Colonnes qualitatives (object)**




In [ ]:
X_train['hydro_station_code'] = X_train['hydro_station_code'].astype('category')
X_test['hydro_station_code'] = X_test['hydro_station_code'].astype('category')
qualitative_columns = ['hydro_status_label', 'hydro_method_label', 'hydro_qualification_label']
X_train[qualitative_columns] = X_train[qualitative_columns].astype('category')
X_test[qualitative_columns] = X_test[qualitative_columns].astype('category')



In [ ]:
# Identifier les valeurs uniques et leur fréquence dans "hydro_hydro_quantity_elab"
hydro_quantity_values = X_train['hydro_hydro_quantity_elab'].value_counts(dropna=False)

# Vérifier les valeurs manquantes
hydro_quantity_nan = X_train['hydro_hydro_quantity_elab'].isna().sum()

# Afficher les résultats
print("Fréquence des valeurs uniques :")
print(hydro_quantity_values)

print("\nNombre de valeurs manquantes :")
print(hydro_quantity_nan)


In [ ]:
X_train['hydro_hydro_quantity_elab'] = X_train['hydro_hydro_quantity_elab'].astype('category')
X_test['hydro_hydro_quantity_elab'] = X_test['hydro_hydro_quantity_elab'].astype('category')



In [ ]:
# Analyse globale pour "hydro_observation_result_elab"
global_stats = X_train['hydro_observation_result_elab'].describe()
global_nan = X_train['hydro_observation_result_elab'].isna().sum()

print("Analyse globale pour 'hydro_observation_result_elab':")
print(global_stats)
print(f"\nNombre de valeurs manquantes : {global_nan}")

# Analyse par type de "hydro_hydro_quantity_elab" (QmJ et QmM)
grouped_stats = X_train.groupby('hydro_hydro_quantity_elab')['hydro_observation_result_elab'].describe()
grouped_nan = X_train.groupby('hydro_hydro_quantity_elab')['hydro_observation_result_elab'].apply(lambda x: x.isna().sum())

print("\nAnalyse par type de 'hydro_hydro_quantity_elab':")
print(grouped_stats)
print("\nNombre de valeurs manquantes par type de 'hydro_hydro_quantity_elab':")
print(grouped_nan)


--> hydro_hydro_quantity_elab ne correspond qu'au mode de calcul et n'affecte pas l'échelle de donnée entre QmJ et QmM 

**Dates (object)**

Description : Date de l’observation élaborée.

Recommandation :
Convertir en datetime pour permettre une analyse temporelle.

In [ ]:
X_train['hydro_observation_date_elab'] = pd.to_datetime(X_train['hydro_observation_date_elab'], errors='coerce')
X_test['hydro_observation_date_elab'] = pd.to_datetime(X_test['hydro_observation_date_elab'], errors='coerce')


**Colonnes numériques**

In [ ]:
hydro_quantitative.describe()

### 1.4 Prelev variables

In [ ]:
# Filtrer les colonnes commençant par "prelev_"
prelev_columns = [col for col in X_train.columns if col.startswith('prelev_')]

# Vérifier les types des colonnes "prelev_"
prelev_types = X_train[prelev_columns].dtypes

# Afficher les résultats
prelev_types


**Colonnes qualitatives (object)**

In [ ]:
# Identifier les colonnes descriptives (object) commençant par "prelev_"
prelev_object_cols = X_train[[col for col in prelev_columns if X_train[col].dtype == 'object']]

# Convertir les colonnes en type "category"
X_train[prelev_object_cols.columns] = X_train[prelev_object_cols.columns].astype('category')
X_test[prelev_object_cols.columns] = X_test[prelev_object_cols.columns].astype('category')



**Variables quantitatives (float64)**


In [ ]:
# Calculer les statistiques descriptives pour les colonnes quantitatives "prelev_"
prelev_quantitative = X_train[prelev_columns].select_dtypes(include=['float64', 'int64']).describe()

prelev_quantitative

In [ ]:
# Identifier les colonnes contenant "commune_code_insee" dans "prelev_"
insee_columns = [col for col in prelev_columns if 'commune_code_insee' in col]

# Convertir les colonnes INSEE en type "category"
X_train[insee_columns] = X_train[insee_columns].astype('category')
X_test[insee_columns] = X_test[insee_columns].astype('category')


# Vérifier le résultat de la conversion
insee_converted_types = X_train[insee_columns].dtypes
insee_converted_types


### 1.5 INSEE variables

In [ ]:
# Identifier les colonnes commençant par "insee"
insee_columns = [col for col in X_train.columns if col.startswith('insee')]

# Vérifier les types des colonnes "insee"
insee_types = X_train[insee_columns].dtypes

# Résultat des types avant et après conversion
print_full(insee_types)


In [ ]:
# Colonnes "insee" à convertir en type float
insee_columns_to_convert = ['insee_%_agri', 'insee_med_living_level', 'insee_%_ind', 'insee_%_const']

# Conversion en type float
X_train[insee_columns_to_convert] = X_train[insee_columns_to_convert].apply(pd.to_numeric, errors='coerce')
X_test[insee_columns_to_convert] = X_test[insee_columns_to_convert].apply(pd.to_numeric, errors='coerce')


# Vérifier les types après conversion
print("Types des colonnes après conversion en float :")
print(X_train[insee_columns_to_convert].dtypes)


### 1.6 Distance variables

In [ ]:
# Identifier les colonnes contenant "distance_"
distance_columns = [col for col in X_train.columns if 'distance_' in col]

# Vérifier les types des colonnes "distance_"
distance_types = X_train[distance_columns].dtypes

# Calculer les statistiques descriptives pour les colonnes "distance_"
distance_stats = X_train[distance_columns].describe()

# Afficher les résultats
distance_types, distance_stats


## II. Suppression de données récurentes / inutiles

In [ ]:
print(X_train.shape)
print(X_test.shape)

#### 2.1 Analyse des variables : Piezo

| **Num** | **Variable**                   | **Description**                                                               | **Décision**     | **Justification**                                                                                                                                   |
|---------|---------------------------------|-------------------------------------------------------------------------------|-----------------|---------------------------------------------------------------------------------------------------------------------------------------------------|
| 1       | piezo_station_department_code  | French department code.                                                      | À conserver     | Utile pour les analyses régionales et la mise en correspondance avec d'autres données géographiques.                                              |
| 2       | piezo_station_update_date      | Date of the last station update.                                             | À Supprimer     |  pas d'utilité                                                          |
| 3       | piezo_station_investigation_depth | Depth of the probe that measures the water table level.                      | À conserver     | Influence directe sur la profondeur de la nappe phréatique et ses variations.                                                                     |
| 4       | piezo_station_department_name  | Name of the department where the watertable station is located.              | À supprimer      | Redondant avec `piezo_station_department_code`.                                                                                                    |
| 5       | piezo_station_commune_code_insee | INSEE code of the city/municipality where the station is located.            | À conserver     | Référence clé pour les analyses géographiques.                                                                                                    |
| 6       | piezo_station_pe_label         | Name of the piezometric table.                                               | À supprimer      | Non nécessaire pour la modélisation, utilisé comme identifiant textuel.                                                                           |
| 7       | piezo_station_bdlisa_codes     | Hydrogeological entity codes (BDLISA).                                       | À conserver     | Critique pour lier les entités géologiques aux mesures des nappes phréatiques.                                                                    |
| 8       | piezo_station_altitude         | NGF altitude of the ground level at the station.                             | À conserver     | Essentiel pour évaluer les variations des niveaux phréatiques en fonction de l'altitude.                                                          |
| 9       | piezo_station_bss_code         | National station code(s) (old BSS codes).                                    | À supprimer      | Redondant avec `piezo_bss_code`.                                                                                                                  |
| 10      | piezo_station_commune_name     | Name of the city/municipality where the station is located.                  | À supprimer      | Non nécessaire pour la modélisation.                                                                                                              |
| 11      | piezo_station_longitude        | Longitude of the station.                                                    | À conserver     | Indispensable pour les analyses géospatiales.                                                                                                     |
| 12      | piezo_station_latitude         | Latitude of the station.                                                     | À conserver     | Indispensable pour les analyses géospatiales.                                                                                                     |
| 13      | piezo_station_bss_id           | National station code(s) (new BSS codes).                                    | À conserver     | Référence unique pour enrichir les données via des correspondances avec BDLISA.                                                                            |
| 14      | piezo_bss_code                 | National station code(s) (old BSS codes).                                    | À supprimer    | Redondant                                                                   |
| 15      | piezo_measurement_date         | Date of the measurement.                                                     | À conserver     | Essentiel pour analyser les tendances temporelles des niveaux phréatiques.                                                                        |
| 16      | piezo_groundwater_level_category | Water table level.                                                           | À conserver     | Target de la modélisation.                                                                                                                        |
| 17      | piezo_obtention_mode           | Method of obtaining the measurement.                                         | À conserver     | Permet de distinguer les valeurs mesurées des valeurs estimées ou corrigées.                                                                       |
| 18      | piezo_status                   | Status of the measurement.                                                   | À conserver     | Indique la qualité et la validation des données.                                                                                                  |
| 19      | piezo_qualification            | Qualification of the measurement.                                            | À conserver     | Critique pour évaluer la fiabilité des mesures.                                                                                                   |
| 20      | piezo_continuity_code          | Code of the point/measurement's continuity.                                  | À supprimer      | Redondant avec `piezo_continuity_name`.                                                                                                           |
| 21      | piezo_continuity_name          | Name of the point/measurement's continuity.                                  | À conserver     | Utile pour comprendre les chroniques des mesures phréatiques.                                                                                     |
| 22      | piezo_producer_code            | Code of the producer of the data.                                            | À supprimer      | Peu pertinent pour le modèle.                                                                                                                     |
| 23      | piezo_producer_name            | Name of the producer of the data.                                            | À supprimer      | Peu pertinent pour le modèle.                                                                                                                     |
| 24      | piezo_measure_nature_code      | Code of nature of the measurement.                                           | À conserver     | Utile pour analyser les niveaux phréatiques influencés ou non par les activités humaines.                                                         |
| 25      | piezo_measure_nature_name      | Nature of the measurement.                                                   | À conserver     | Complète `piezo_measure_nature_code` pour des analyses plus détaillées.                         |



In [ ]:
piezo_columns_to_drop = [
    "piezo_station_department_name", "piezo_station_pe_label", 
    "piezo_station_bss_code", "piezo_station_commune_name", 
    "piezo_bss_code", "piezo_continuity_code", 
    "piezo_producer_code", "piezo_producer_name", "piezo_station_update_date"
]

# Supprimer les colonnes inutiles
X_train.drop(columns=piezo_columns_to_drop, inplace=True)
X_test.drop(columns=piezo_columns_to_drop, inplace=True)

# Vérification
print("Colonnes supprimées :", len(piezo_columns_to_drop))


In [ ]:
print(X_train.shape)
print(X_test.shape)

#### 2.2 Analyse des variables : Meteo

| **Num** | **Variable**                      | **Description**                                                                 | **Décision**     | **Justification**                                                                                                                                  |
|---------|----------------------------------|-------------------------------------------------------------------------------|-----------------|--------------------------------------------------------------------------------------------------------------------------------------------------|
| 1       | meteo_cloudiness_height         | Minimum height of the layer >300m with maximum cloudiness > 4/8               | À conserver     | Peut refléter une condition atmosphérique impactant les nappes phréatiques.                                                                      |
| 2       | meteo_if_fog                   | Fog occurrence                                                                | À supprimer      | Faible lien direct avec le niveau de la nappe phréatique.                                                                                         |
| 3       | meteo_if_mist                  | Haze occurrence                                                               | À supprimer      | Faible corrélation attendue avec la Target.                                                                                                       |
| 4       | meteo_frost_duration           | Freezing duration                                                             | À conserver      | Influence potentielle sur l’infiltration de l’eau dans le sol.                                                                                   |
| 5       | meteo_wetting_duration         | Wetting duration                                                              | À conserver      | Impact possible sur la recharge des nappes phréatiques.                                                                                          |
| 6       | meteo_humidity_duration_below_40% | Humidity <= 40% duration                                                      | À conserver      | Peut influencer l’évaporation et l’infiltration des eaux.                                                                                         |
| 7       | meteo_humidity_duration_above_80% | Humidity >= 80% duration                                                      | À conserver      | Impact potentiel sur la recharge des nappes.                                                                                                      |
| 8       | meteo_radiation_direct         | Direct radiation level                                                        | À supprimer      | Influence indirecte et négligeable sur la Target.                                                                                                 |
| 9       | meteo_wind_direction_max_inst  | Maximum instant wind direction                                                | À supprimer      | Peu de pertinence pour la prédiction du niveau de la nappe phréatique.                                                                            |
| 10      | meteo_wind_direction_max_inst_2m | Maximum instant direction at 2 meters                                         | À supprimer      | Peu de pertinence pour la Target.                                                                                                                 |
| 11      | meteo_wind_direction_max_avg   | Average maximum wind direction over 10 minutes                                | À supprimer      | Lien indirect, peut être négligeable pour la recharge des nappes.                                                                                 |
| 12      | meteo_if_lightning             | Lightning occurrence                                                          | À supprimer      | Faible pertinence pour le modèle.                                                                                                                 |
| 13      | meteo_evapotranspiration_grid  | Evapotranspiration potential calculated at the closest grid point             | À conserver      | Impact direct sur la recharge des nappes phréatiques.                                                                                             |
| 14      | meteo_evapotranspiration_Monteith | Monteith evapotranspiration                                                   | À conserver      | Indicateur clé des pertes d’eau via l’évaporation et la transpiration.                                                                             |
| 15      | meteo_wind_speed_avg_2m        | Average wind speed at 2 meters                                                | À supprimer      | Corrélation faible ou indirecte avec la Target.                                                                                                   |
| 16      | meteo_wind_speed_avg_10m       | Average wind speed at 10 meters                                               | À supprimer      | Peu pertinent pour la prédiction.                                                                                                                 |
| 17      | meteo_if_smoke                 | Occurrence of smoke                                                           | À supprimer      | Aucune pertinence directe pour le modèle.                                                                                                         |
| 18      | meteo_wind_max                 | Maximum instant wind speed                                                    | À supprimer      | Peu d’impact sur la recharge des nappes phréatiques.                                                                                              |
| 19      | meteo_wind_max_2m              | Maximum instant wind speed at 2 meters                                        | À supprimer      | Non pertinent pour la Target.                                                                                                                     |
| 20      | meteo_wind_max_3s              | Maximum 3-second instant wind speed within an hour                            | À supprimer      | Faible pertinence pour la recharge des nappes.                                                                                                    |
| 21      | meteo_wind_avg                 | Average maximum wind speed over 10 minutes                                    | À supprimer      | Corrélation indirecte et négligeable.                                                                                                             |
| 22      | meteo_if_frost                 | Occurrence of frost                                                           | À conserver      | Impact potentiel sur l’infiltration de l’eau.                                                                                                     |
| 23      | meteo_radiation                | Global radiation level                                                        | À supprimer      | Influence indirecte et négligeable.                                                                                                               |
| 24      | meteo_if_hail                  | Occurrence of hail                                                            | À supprimer      | Non pertinent pour la prédiction de la Target.                                                                                                    |
| 25      | meteo_if_sleet                 | Occurrence of sleet                                                           | À supprimer      | Aucune corrélation directe avec la recharge des nappes phréatiques.                                                                               |
| 26      | meteo_snow_height              | Snowfall height over 24 hours                                                 | À conserver      | Peut contribuer à la recharge en eau via la fonte des neiges.                                                                                     |
| 27      | meteo_time_tn                  | Time of minimum temperature in shelter                                        | À conserver      | Influence sur l’infiltration et l’évaporation.                                                                                                    |
| 28      | meteo_time_tx                  | Time of maximum temperature in shelter                                        | À conserver      | Impact potentiel sur l’évaporation et la température du sol.                                                                                      |
| 29      | meteo_time_humidity_min        | Time of minimum humidity                                                      | À supprimer      | Pertinence faible ou indirecte.                                                                                                                   |
| 30      | meteo_time_humidity_max        | Time of maximum humidity                                                      | À supprimer      | Faible lien avec la recharge des nappes phréatiques.                                                                                              |
| 31      | meteo_time_wind_max            | Time of maximum instant wind                                                  | À supprimer      | Non pertinent pour le modèle.                                                                                                                     |
| 32      | meteo_time_wind_max_2m         | Time of maximum instantaneous wind at 2 meters                                | À supprimer      | Non pertinent pour la Target.                                                                                                                     |
| 33      | meteo_time_wind_max_3s         | 3-second instantaneous wind maximum                                           | À supprimer      | Faible pertinence pour la recharge des nappes.                                                                                                    |
| 34      | meteo_time_wind_avg            | Time of average maximum wind speed over 10 minutes                            | À supprimer      | Corrélation indirecte et faible.                                                                                                                  |
| 35      | meteo_id                       | Weather station ID                                                            | À conserver      | Référence clé pour les données météorologiques.                                                                                                   |
| 36      | meteo_radiation_IR             | Total hourly infrared radiation                                               | À supprimer      | Peu d’influence directe sur la Target.                                                                                                            |
| 37      | meteo_sunshine_duration        | Sunshine duration                                                             | À conserver      | Impact potentiel sur l’évaporation et l’infiltration.                                                                                             |
| 38      | meteo_cloudiness               | Maximum cloudiness above 4/8 of a layer below 300 m                           | À conserver      | Indicateur clé des conditions atmosphériques. 
| 39      | meteo_longitude                  | Weather station longitude                                                    | À conserver     | Utilisée pour les analyses géospatiales et pour la mise en correspondance des stations avec les données piezo.                                    |
| 40      | meteo_latitude                   | Weather station latitude                                                     | À conserver     | Idem que `meteo_longitude`, essentielle pour l’analyse géospatiale.                                                                               |
| 41      | meteo_cloudiness                 | Maximum cloudiness above 4/8 of a layer below 300 m                           | À conserver     | Indicateur clé des conditions météorologiques pouvant influencer les précipitations et la recharge des nappes.                                    |
| 42      | meteo_if_snow                    | Occurrence of snow                                                           | À conserver     | Concerne les précipitations solides qui peuvent contribuer à la recharge après fonte.                                                             |
| 43      | meteo_snow_thickness_6h          | Total snow depth recorded at 0600 UTC                                        | À conserver     | Permet une estimation des précipitations solides contribuant à la recharge des nappes phréatiques.                                                |
| 44      | meteo_snow_thickness_max         | Maximum of hourly total snow depths                                          | À conserver     | Complète `meteo_snow_thickness_6h` pour analyser les précipitations solides.                                                                      |
| 45      | meteo_name                       | Weather station name                                                        | À supprimer      | Utilisé uniquement comme identifiant textuel, pas pertinent pour le modèle de prédiction.                                                         |
| 46      | meteo_if_thunderstorm            | Occurrence of storm                                                         | À supprimer      | Peu d’influence directe sur la recharge des nappes phréatiques.                                                                                   |
| 47      | meteo_pressure_avg               | Average sea-level pressure                                                  | À conserver     | Peut influencer les conditions météorologiques locales, notamment les précipitations.                                                             |
| 48      | meteo_pressure_max               | Minimum sea-level pressure                                                  | À conserver     | Indicateur complémentaire pour évaluer les conditions météorologiques, notamment les tempêtes ou dépressions.                                     |
| 49      | meteo_if_dew                     | Occurrence of dew                                                           | À supprimer      | Corrélation faible avec la recharge des nappes phréatiques.                                                                                       |
| 50      | meteo_rain_height                | Rain height                                                                 | À conserver     | Indicateur direct des précipitations contribuant à la recharge des nappes phréatiques.                                                            |
| 51      | meteo_sunshine_%                 | Sunshine ratio                                                              | À conserver     | Influence potentielle sur l’évaporation et les conditions climatiques locales.                                                                    |
| 52      | meteo_if_snow_ground             | Occurrence of snow-covered ground                                           | À conserver     | Pertinent pour analyser les périodes où l'eau peut être stockée sous forme solide avant infiltration.                                              |
| 53      | meteo_amplitude_tn_tx            | Amplitude between min and max temperatures                                  | À conserver     | Permet d’estimer les variations thermiques pouvant influencer l’infiltration de l’eau.                                                             |
| 54      | meteo_temperature_avg            | Average shelter temperature                                                 | À conserver     | Influence directe sur l’évaporation et les conditions d’infiltration.                                                                             |
| 55      | meteo_temperature_avg_threshold  | Average shelter temperature from (min+max)/2                                | À conserver     | Indicateur complémentaire à `meteo_temperature_avg`.                                                                                              |
| 56      | meteo_temperature_min            | Minimum shelter temperature                                                 | À conserver     | Pertinent pour analyser les impacts des basses températures sur l’infiltration ou le gel des sols.                                                |
| 57      | meteo_temperature_min_50cm       | Minimum temperature at +50cm                                                | À supprimer      | Peu pertinent, déjà couvert par `meteo_temperature_min`.                                                                                           |
| 58      | meteo_temperature_min_ground     | Minimum temperature at +10cm                                                | À conserver     | Pertinent pour estimer les impacts directs sur la surface du sol et l’infiltration.                                                               |
| 59      | meteo_temperature_avg_tntm       | Average of min and max temperatures                                         | À supprimer      | Redondant avec `meteo_temperature_avg` et `meteo_temperature_avg_threshold`.                                                                       |
| 60      | meteo__pressure_saturation_avg   | Average vapor tension                                                      | À supprimer      | Faible impact attendu sur la recharge des nappes phréatiques.                                                                                     |
| 61      | meteo_humidity_avg               | Average relative humidity                                                  | À conserver     | Influence directe sur l’évaporation et les conditions météorologiques.                                                                            |
| 62      | meteo_humidity_min               | Minimum relative humidity                                                  | À supprimer      | Peu pertinent, `meteo_humidity_avg` est suffisant.                                                                                                |
| 63      | meteo_radiation_UV               | Ultraviolet radiation level                                                | À supprimer      | Peu pertinent pour l’influence sur la Target.                                                                                                     |
| 64      | meteo_radiation_UV_max           | Maximum hourly UV index                                                    | À supprimer      | Peu pertinent pour le modèle.                                                                                                                     |
| 65      | meteo_humidity_max               | Maximum relative humidity                                                  | À supprimer      | Redondant avec `meteo_humidity_avg`.                                                                                                              |
| 66      | meteo_if_black_ice               | Occurrence of black ice                                                    | À supprimer      | Non pertinent pour la recharge des nappes phréatiques.                        


**--> à décider ensemble !!!**

In [ ]:
meteo_columns_to_drop = [
    "meteo_if_fog", "meteo_if_mist", "meteo_radiation_direct", 
    "meteo_wind_direction_max_inst", "meteo_wind_direction_max_inst_2m", 
    "meteo_wind_direction_max_avg", "meteo_if_lightning", 
    "meteo_wind_speed_avg_2m", "meteo_wind_speed_avg_10m", 
    "meteo_if_smoke", "meteo_wind_max", "meteo_wind_max_2m", 
    "meteo_wind_max_3s", "meteo_wind_avg", "meteo_radiation", 
    "meteo_if_hail", "meteo_if_sleet", "meteo_time_humidity_min", 
    "meteo_time_humidity_max", "meteo_time_wind_max", "meteo_time_wind_max_2m", 
    "meteo_time_wind_max_3s", "meteo_time_wind_avg", "meteo_radiation_IR", 
    "meteo_name", "meteo_if_thunderstorm", "meteo_if_dew", 
    "meteo_temperature_min_50cm", "meteo_temperature_avg_tntm", 
    "meteo__pressure_saturation_avg", "meteo_humidity_min", 
    "meteo_radiation_UV", "meteo_radiation_UV_max", "meteo_humidity_max", 
    "meteo_if_black_ice"
]

# Supprimer les colonnes inutiles
X_train.drop(columns=meteo_columns_to_drop, inplace=True)
X_test.drop(columns=meteo_columns_to_drop, inplace=True)


In [ ]:
print(X_train.shape)
print(X_test.shape)

#### 2.3 Analyse des variables : Hydro

| **Variable**                     | **Description**                                                                                   | **Décision**             | **Justification**                                                                                                              |
|----------------------------------|---------------------------------------------------------------------------------------------------|---------------------------|------------------------------------------------------------------------------------------------------------------------------|
| `hydro_station_code`             | Hydrometric station code                                                                          | **Supprimer**            | Les identifiants spécifiques aux stations hydrologiques ne sont pas nécessaires pour la prédiction des niveaux piezométriques. |
| `hydro_observation_date_elab`    | Hydrometric observation date                                                                      | **Conserver**            | Permet de synchroniser les données hydrologiques avec d'autres variables temporelles.                                         |
| `hydro_observation_result_elab`  | Value of the hydrometric measurement (flow rate)                                                  | **Conserver**            | Variable quantitative clé pour estimer les apports hydriques contribuant à la recharge de la nappe phréatique.               |
| `hydro_status_code`              | Code of the data's status                                                                         | **Supprimer**            | La qualité des données est déjà reflétée par d'autres indicateurs (ex. qualification).                                        |
| `hydro_status_label`             | Label/Name of the data's status                                                                   | **Supprimer**            | Redondant avec `hydro_status_code`.                                                                                          |
| `hydro_method_code`              | Code of the method for obtaining the hydro observation result                                     | **Supprimer**            | Peu pertinent pour l'analyse, les valeurs hydrométriques restent valides quelle que soit leur méthode d'obtention.           |
| `hydro_method_label`             | Method for obtaining the hydro observation result                                                | **Supprimer**            | Redondant avec `hydro_method_code`.                                                                                          |
| `hydro_qualification_code`       | Code of the quality of the measurement                                                           | **Supprimer**            | Peu utile si les données sont déjà filtrées ou nettoyées en amont.                                                           |
| `hydro_qualification_label`      | Quality of the measurement produced                                                              | **Supprimer**            | Redondant avec `hydro_qualification_code`.                                                                                   |
| `hydro_longitude`                | Longitude of the hydrometric station                                                             | **Supprimer**            | Les coordonnées géographiques des stations hydrométriques ne sont pas nécessaires, la position des stations piezo est suffisante. |
| `hydro_latitude`                 | Latitude of the hydrometric station                                                              | **Supprimer**            | Idem.                                                                                                                        |
| `hydro_hydro_quantity_elab`      | Type of hydrometric quantity developed (e.g., daily/monthly mean discharge)                      | **Conserver**            | Utile pour différencier les séries temporelles journalières et mensuelles et ajuster les modèles en conséquence.             |

---


**--> à décider ensemble !!!**

In [ ]:
# Liste des colonnes inutiles
hydro_columns_to_drop = [
    'hydro_station_code', 'hydro_status_code', 'hydro_status_label',
    'hydro_method_code', 'hydro_method_label', 'hydro_qualification_code',
    'hydro_qualification_label', 'hydro_longitude', 'hydro_latitude'
]

# Suppression dans les datasets
X_train = X_train.drop(columns=hydro_columns_to_drop, errors='ignore')
X_test = X_test.drop(columns=hydro_columns_to_drop, errors='ignore')

print(f"Colonnes supprimées (Hydrologie) : {len(hydro_columns_to_drop)}")

In [ ]:
print(X_train.shape)
print(X_test.shape)

#### 2.4 Analyse des variables : Prelev

| **Variable**                             | **Description**                                                                                      | **Décision**             | **Justification**                                                                                                                                                            |
|------------------------------------------|------------------------------------------------------------------------------------------------------|---------------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `prelev_structure_code_0`                | ID of the water withdrawal facility                                                                  | **Supprimer**            | Non pertinent pour la prédiction des niveaux piezométriques, car ces identifiants ne contribuent pas directement à l’analyse des variables hydrologiques globales.         |
| `prelev_volume_0`                        | Maximum water withdrawal volume (largest annual withdrawal)                                          | **Conserver**            | Variable clé pour estimer l’impact des prélèvements sur les niveaux piezométriques.                                                                                        |
| `prelev_usage_label_0`                   | Goal/usage of the withdrawal                                                                         | **Conserver**            | Permet d’expliquer la répartition des prélèvements selon les usages (eau potable, énergie, irrigation, etc.).                                                              |
| `prelev_volume_obtention_mode_label_0`   | Type of measurement                                                                                 | **Supprimer**            | Peu pertinent pour l’analyse, car la précision du mode d’obtention n’influence pas directement les résultats piezométriques.                                               |
| `prelev_longitude_0`                     | Longitude of the water withdrawal facility                                                          | **Supprimer**            | Déjà supprimée dans les étapes précédentes au profit des coordonnées des stations piezométriques.                                                                           |
| `prelev_latitude_0`                      | Latitude of the water withdrawal facility                                                           | **Supprimer**            | Idem.                                                                                                                                                                       |
| `prelev_commune_code_insee_0`            | City/municipality code where the withdrawal was made                                                | **Supprimer**            | Déjà remplacée par `piezo_station_commune_code_insee` comme référence unique.                                                                                               |
| `prelev_structure_code_1`                | ID of the water withdrawal facility                                                                  | **Supprimer**            | Idem que pour `prelev_structure_code_0`.                                                                                                                                   |
| `prelev_volume_1`                        | Maximum water withdrawal volume (second-largest annual withdrawal)                                  | **Conserver**            | Variable importante pour évaluer l’impact cumulé des prélèvements sur les niveaux piezométriques.                                                                          |
| `prelev_usage_label_1`                   | Goal/usage of the withdrawal                                                                         | **Conserver**            | Idem que pour `prelev_usage_label_0`.                                                                                                                                      |
| `prelev_volume_obtention_mode_label_1`   | Type of measurement                                                                                 | **Supprimer**            | Idem que pour `prelev_volume_obtention_mode_label_0`.                                                                                                                      |
| `prelev_longitude_1`                     | Longitude of the water withdrawal facility                                                          | **Supprimer**            | Idem que pour `prelev_longitude_0`.                                                                                                                                        |
| `prelev_latitude_1`                      | Latitude of the water withdrawal facility                                                           | **Supprimer**            | Idem que pour `prelev_latitude_0`.                                                                                                                                         |
| `prelev_commune_code_insee_1`            | City/municipality code where the withdrawal was made                                                | **Supprimer**            | Idem que pour `prelev_commune_code_insee_0`.                                                                                                                               |
| `prelev_structure_code_2`                | ID of the water withdrawal facility                                                                  | **Supprimer**            | Idem que pour `prelev_structure_code_0`.                                                                                                                                   |
| `prelev_volume_2`                        | Maximum water withdrawal volume (third-largest annual withdrawal)                                   | **Conserver**            | Idem que pour `prelev_volume_0`.                                                                                                                                          |
| `prelev_usage_label_2`                   | Goal/usage of the withdrawal                                                                         | **Conserver**            | Idem que pour `prelev_usage_label_0`.                                                                                                                                      |
| `prelev_volume_obtention_mode_label_2`   | Type of measurement                                                                                 | **Supprimer**            | Idem que pour `prelev_volume_obtention_mode_label_0`.                                                                                                                      |
| `prelev_longitude_2`                     | Longitude of the water withdrawal facility                                                          | **Supprimer**            | Idem que pour `prelev_longitude_0`.                                                                                                                                        |
| `prelev_latitude_2`                      | Latitude of the water withdrawal facility                                                           | **Supprimer**            | Idem que pour `prelev_latitude_0`.                                                                                                                                         |
| `prelev_commune_code_insee_2`            | City/municipality code where the withdrawal was made                                                | **Supprimer**            | Idem que pour `prelev_commune_code_insee_0`.                                                                                                                               |
| `prelev_other_volume_sum`                | Sum of other water withdrawals                                                                      | **Conserver**            | Permet de capturer l’impact global des prélèvements en dehors des trois principaux prélèvements identifiés individuellement.                                               |

---

In [ ]:
# Liste des colonnes inutiles
prelev_columns_to_drop = [
    'prelev_structure_code_0', 'prelev_volume_obtention_mode_label_0', 'prelev_longitude_0', 'prelev_latitude_0', 'prelev_commune_code_insee_0',
    'prelev_structure_code_1', 'prelev_volume_obtention_mode_label_1', 'prelev_longitude_1', 'prelev_latitude_1', 'prelev_commune_code_insee_1',
    'prelev_structure_code_2', 'prelev_volume_obtention_mode_label_2', 'prelev_longitude_2', 'prelev_latitude_2', 'prelev_commune_code_insee_2'
]

# Suppression dans les datasets
X_train = X_train.drop(columns=prelev_columns_to_drop, errors='ignore')
X_test = X_test.drop(columns=prelev_columns_to_drop, errors='ignore')

print(f"Colonnes supprimées (Prélèvement) : {len(prelev_columns_to_drop)}")

In [ ]:
print(X_train.shape)
print(X_test.shape)

#### 2.5 Analyse des variables : INSEE


| **Variable**              | **Description**                                           | **Décision**  | **Justification**                                                                                           |
|---------------------------|-----------------------------------------------------------|---------------|-------------------------------------------------------------------------------------------------------------|
| `insee_%_agri`            | Percentage of the working population in agriculture       | **Conserver** | Donnée pertinente pour évaluer l’impact des activités agricoles sur les prélèvements d’eau et les niveaux piezométriques. |
| `insee_pop_commune`       | Number of inhabitants of the municipality                 | **Conserver** | Variable clé pour estimer la demande en eau, qui peut influencer les prélèvements et les niveaux piezométriques. |
| `insee_med_standard_living` | Median standard of living                                | **Conserver** | Peut refléter indirectement le développement économique et les pressions sur les ressources en eau.         |
| `insee%ind`               | Percentage of the working population in the industry sector | **Conserver** | Reflète l’impact potentiel des activités industrielles sur la consommation et les prélèvements d’eau.       |
| `insee_%_const`           | Percentage of the working population in the construction sector | **Conserver** | Peut refléter les impacts liés aux besoins en eau pour les projets de construction, notamment pour l’irrigation ou le refroidissement. |

---


#### 2.6 Analyse des variables : Distance

| **Variable**             | **Description**                                      | **Décision**  | **Justification**                                                                                     |
|--------------------------|------------------------------------------------------|---------------|-------------------------------------------------------------------------------------------------------|
| `distance_piezo_meteo`   | Distance between the piezometric and weather station | **Conserver** | Permet de mesurer l’influence potentielle des données météorologiques sur les observations piezométriques. |
| `distance_hydro_meteo`   | Distance between the piezometric and hydrological station | **Conserver** | Peut aider à évaluer la cohérence spatiale des données hydrologiques par rapport aux stations piezométriques. |

---



In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
variables_utiles = np.array(X_train.columns)
print(variables_utiles)

#### --> 68 Variables à conserver

['row_index' 'piezo_station_department_code' 'piezo_station_update_date'
 'piezo_station_investigation_depth' 'piezo_station_commune_code_insee'
 'piezo_station_pe_label' 'piezo_station_bdlisa_codes'
 'piezo_station_altitude' 'piezo_station_bss_code'
 'piezo_station_longitude' 'piezo_station_latitude' 'piezo_bss_code'
 'piezo_measurement_date' 'piezo_obtention_mode' 'piezo_status'
 'piezo_qualification' 'piezo_continuity_code' 'piezo_continuity_name'
 'piezo_producer_code' 'piezo_producer_name' 'piezo_measure_nature_code'
 'piezo_measure_nature_name' 'meteo_id' 'meteo_name' 'meteo_latitude'
 'meteo_longitude' 'meteo_altitude' 'meteo_date' 'meteo_rain_height'
 'meteo_DRR' 'meteo_temperature_min' 'meteo_temperature_max'
 'meteo_temperature_avg' 'meteo_temperature_avg_threshold'
 'meteo_temperature_min_ground' 'meteo_temperature_min_50cm'
 'meteo_frost_duration' 'meteo_amplitude_tn_tx'
 'meteo_temperature_avg_tntm' 'meteo_pression_maxi' 'meteo_time_wind_avg'
 'meteo_humidity_min' 'meteo_humidity_max'
 'meteo_humidity_duration_below_40%' 'meteo_humidity_duration_above_80%'
 'meteo__pressure_saturation_avg' 'meteo_wetting_duration'
 'meteo_humidity_avg' 'meteo_sunshine_%' 'meteo_radiation_UV_max'
 'meteo_cloudiness_height' 'meteo_if_snow' 'meteo_if_dew'
 'meteo_if_black_ice' 'meteo_if_snow_ground' 'meteo_if_frost'
 'meteo_if_smoke' 'meteo_evapotranspiration_Monteith'
 'meteo_evapotranspiration_grid' 'meteo_radiation_UV' 'meteo_snow_height'
 'meteo_snow_thickness_max' 'meteo_snow_thickness_6h'
 'distance_piezo_meteo' 'hydro_observation_date_elab'
 'hydro_observation_result_elab' 'hydro_hydro_quantity_elab'
 'distance_piezo_hydro' 'prelev_volume_0' 'prelev_usage_label_0'
 'prelev_volume_1' 'prelev_usage_label_1' 'prelev_volume_2'
 'prelev_usage_label_2' 'prelev_other_volume_sum' 'insee_%_agri'
 'insee_pop_commune' 'insee_med_living_level' 'insee_%_ind'
 'insee_%_const']

## III. Traitement des NaN

### 3.1 Analyse

In [ ]:
# Redéfinir les étapes pour afficher les colonnes avec des NaN

# Vérifier les NaN dans X_train
nan_counts_train = X_train.isna().sum()
nan_counts_train = nan_counts_train[nan_counts_train > 0].sort_values(ascending=False)

# Vérifier les NaN dans X_test
nan_counts_test = X_test.isna().sum()
nan_counts_test = nan_counts_test[nan_counts_test > 0].sort_values(ascending=False)


train_nan_df = pd.DataFrame({'Column': nan_counts_train.index, 'Number of NaN': nan_counts_train.values})
test_nan_df = pd.DataFrame({'Column': nan_counts_test.index, 'Number of NaN': nan_counts_test.values})

train_nan_df, test_nan_df


In [ ]:
# Vérifier les NaN dans X_train
nan_counts_train = X_train.isna().sum()
nan_counts_train = nan_counts_train[nan_counts_train > 0].sort_values(ascending=False)

# Vérifier les NaN dans X_test
nan_counts_test = X_test.isna().sum()
nan_counts_test = nan_counts_test[nan_counts_test > 0].sort_values(ascending=False)

# Afficher les résultats pour X_train
print("Variables avec des NaN dans X_train :")
print(nan_counts_train)
print(nan_counts_train.shape)

# Afficher les résultats pour X_test
print("\nVariables avec des NaN dans X_test :")
print(nan_counts_test)
print(nan_counts_test.shape)


In [ ]:
# Identifier les colonnes avec des NaN dans X_test mais pas dans X_train
nan_in_test_not_in_train = set(nan_counts_test.index) - set(nan_counts_train.index)

# Afficher les résultats
print("Variables avec des NaN uniquement dans X_test et pas dans X_train :")
print(nan_in_test_not_in_train)


### 3.2 Suppressions des plus de 70% de NaN

In [ ]:
# Définir le seuil de 70% de NaN
threshold = 0.7

# Identifier les colonnes avec plus de 70% de NaN dans X_train
columns_to_drop = X_train.columns[X_train.isna().mean() > threshold]

# Vérifier si ces colonnes existent également dans X_test
columns_to_drop_test = X_test.columns.intersection(columns_to_drop)

# Supprimer les colonnes dans X_train et X_test
X_train.drop(columns=columns_to_drop, inplace=True)
X_test.drop(columns=columns_to_drop_test, inplace=True)

# Vérification après suppression
print(f"Colonnes supprimées : {list(columns_to_drop)}")
print(f"Taille X_train : {X_train.shape}")
print(f"Taille X_test : {X_test.shape}")


In [ ]:
# Vérifier les NaN dans X_train
nan_counts_train = X_train.isna().sum()
nan_counts_train = nan_counts_train[nan_counts_train > 0].sort_values(ascending=False)

# Vérifier les NaN dans X_test
nan_counts_test = X_test.isna().sum()
nan_counts_test = nan_counts_test[nan_counts_test > 0].sort_values(ascending=False)

# Afficher les résultats pour X_train
print("Variables avec des NaN dans X_train :")
print(nan_counts_train)
print(nan_counts_train.shape)

# Afficher les résultats pour X_test
print("\nVariables avec des NaN dans X_test :")
print(nan_counts_test)
print(nan_counts_test.shape)

### 3.3 Par type de Variable

#### 3.3.1 Piezo

In [ ]:
# Identifier les colonnes piezo ayant des NaN
piezo_columns = [col for col in X_train.columns if col.startswith('piezo')]
piezo_nan_counts = X_train[piezo_columns].isna().sum().sort_values(ascending=False)

# Affichage des variables piezo avec des NaN
print("Variables 'piezo' avec des NaN :")
print(piezo_nan_counts[piezo_nan_counts > 0])


piezo_measure_nature_code            112386   
piezo_measure_nature_name            112386   

--> on mets les valeurs décrites dans le fichier en cas de donnée manquante (0 et unknown)

In [ ]:
# Imputation des valeurs manquantes pour piezo_measure_nature_code
X_train['piezo_measure_nature_code'].fillna("0", inplace=True)
X_test['piezo_measure_nature_code'].fillna("0", inplace=True)
""
# Imputation des valeurs manquantes pour piezo_measure_nature_name
X_train['piezo_measure_nature_name'].fillna('Inconnue', inplace=True)
X_test['piezo_measure_nature_name'].fillna('Inconnue', inplace=True)


In [ ]:
X_train['piezo_station_bdlisa_codes'].value_counts(dropna=False)

In [ ]:
X_train['piezo_station_bdlisa_codes'].dtype

In [ ]:
# Remplacer les NaN dans 'piezo_station_bdlisa_codes' par "Inconnus"
X_train['piezo_station_bdlisa_codes'] = X_train['piezo_station_bdlisa_codes'].cat.add_categories("Inconnus")
X_train['piezo_station_bdlisa_codes'].fillna("Inconnus", inplace=True)

X_test['piezo_station_bdlisa_codes'] = X_test['piezo_station_bdlisa_codes'].cat.add_categories("Inconnus")
X_test['piezo_station_bdlisa_codes'].fillna("Inconnus", inplace=True)


In [ ]:
X_train['piezo_station_investigation_depth'].value_counts(dropna=False)

Remplacement par la médiane

In [ ]:
X_train['piezo_station_investigation_depth'].fillna(X_train['piezo_station_investigation_depth'].median(), inplace=True)


### 3.3.2 Meteo

In [ ]:
# Identifier les colonnes meteo ayant des NaN
meteo_columns = [col for col in X_train.columns if col.startswith('meteo')]
meteo_nan_counts = X_train[meteo_columns].isna().sum().sort_values(ascending=False)

# Affichage des variables meteo avec des NaN
print("Variables 'meteo' avec des NaN :")
print(meteo_nan_counts[meteo_nan_counts > 0])


On regarde la corélation avec la position géographique

### 3.3.4 Prelev

In [ ]:
# Identifier les colonnes prelev ayant des NaN
prelev_columns = [col for col in X_train.columns if col.startswith('prelev')]
prelev_nan_counts = X_train[prelev_columns].isna().sum().sort_values(ascending=False)

# Affichage des variables prelev avec des NaN
print("Variables 'prelev' avec des NaN :")
print(prelev_nan_counts[prelev_nan_counts > 0])


In [ ]:
# Labels pour les usages de prélèvement
prelev_usage_labels = [
    'EAU POTABLE',
    'EAU TURBINEE (barrage)',
    'CANAUX',
    'INDUSTRIE et ACTIVITES ECONOMIQUES (hors irrigation, hors énergie)',
    'IRRIGATION',
    'ENERGIE',
]

# Fonction pour transformer les données des prélèvements
def fix_prelev(df):
    # Ajouter une colonne pour le volume total de prélèvement
    df['prelev_TOTAL'] = (
        df['prelev_volume_0'].fillna(0) +
        df['prelev_volume_1'].fillna(0) +
        df['prelev_volume_2'].fillna(0) +
        df['prelev_other_volume_sum'].fillna(0)
    )
    
    # Créer une colonne par type d'usage
    for usage in prelev_usage_labels:
        df[f'prelev_{usage}'] = (
            (df['prelev_volume_0'].fillna(0) * (df['prelev_usage_label_0'] == usage).astype(int)) +
            (df['prelev_volume_1'].fillna(0) * (df['prelev_usage_label_1'] == usage).astype(int)) +
            (df['prelev_volume_2'].fillna(0) * (df['prelev_usage_label_2'] == usage).astype(int))
        )
    
    # Supprimer les colonnes originales
    df = df.drop(columns=[
        'prelev_usage_label_0', 'prelev_usage_label_1', 'prelev_usage_label_2',
        'prelev_volume_0', 'prelev_volume_1', 'prelev_volume_2','prelev_other_volume_sum'
    ])
    
    return df



In [ ]:
# Appliquer la fonction à X_train et X_test
X_train = fix_prelev(X_train)
X_test = fix_prelev(X_test)

In [ ]:
# Identifier les colonnes prelev ayant des NaN
prelev_columns = [col for col in X_train.columns if col.startswith('prelev')]
prelev_nan_counts = X_train[prelev_columns].isna().sum().sort_values(ascending=False)

# Affichage des variables prelev avec des NaN
print("Variables 'prelev' avec des NaN :")
print(prelev_nan_counts[prelev_nan_counts > 0])